# Simulación de Taxis de NYC por el método Montecarlo

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta

Cargando datos

In [2]:
taxi = pd.read_csv('../../Data/Data_taxi_sim.csv')
fuel = pd.read_parquet('../../Data/Eda_fuel_vehicles.parquet')
electric = pd.read_parquet('../../Data/df_final_electrico01.parquet')
electric_min = pd.read_csv('../../Data/ElectricCarData_Clean.csv')
# fuel_data = pd.read_csv('../../Data/Vehicle Fuel Economy Data.csv')
demanda = pd.read_csv('../../Data/demandaxzona.csv')

Transformaciones

In [ ]:
taxi['time'] = taxi['tpep_dropoff_datetime'] - taxi['tpep_pickup_datetime']
taxi['time'] = taxi['time'].apply(lambda x: int(x.total_seconds() // 60))
taxi = taxi.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)

### Funciones

In [4]:
def rango(df,column:str):
    '''
Esta función devuelve un rango de valores que excluye outliers, la función recibe
    - df - Un dataFrame
    - column - Una columna en str
    La función devuelve una tupla con el intervalo valido
    '''
    statistics = df[column].describe()
    IQR = statistics['75%'] - statistics['25%']
    RI = statistics['25%'] - 1.5 * IQR       # RAngo inferior
    if RI < 0:
        RI = 0
    RS = statistics['75%'] + 1.5 * IQR       # Rango superior
    if RI == RS:
        return (RI-1,RS)
    else:
        return (RI , RS)

In [5]:
def delay(df,hour:int,zone:int):
    '''
    Esta Función simula el tiempo que el taxi se mantiene oscioso por la demanda que existe en la zona haciendo uso de
    el modelo M/M/1 para teoria de colas, se considera el día y la zona en la que se solicita el servicio.

    La función solicita:
    - `df` Dataframe con los datos, `Hour`, `Zone`, `TimeArrival(customers/min)` y `Time4Service(customers/min)`
    - `hour` La hora en la que el taxi esta disponible para un servicio. En valor entero
    - `Zona` La zona en donde se encuentra el taxi para iniciar un nuevo servicio. En valor entero

    La función devuelve el tiempo en minutos que se mantiene ocioso el taxi antes de su siguiente servicio.
    '''
    df = df[(df['Zone'] == zone) & (df['Hour'] == hour)]
    Lambda = df['TimeArrival(customers/min)'].mean()
    Mu = df['Time4Service(customers/min)'].mean()
    if Mu == 0:
        return trip(zone,timedelta(hours=hour,minutes=0))['trip_time']
    else:
        return Lambda ** 2 / (Mu * (Mu - Lambda))

In [74]:
def trip(UpLocationId:int,Hour):
    '''
    Esta función devuelve caracteristicas de la ruta entre dos puntos. La función recibe:
        - UpLocationId. Id de la zona de inicio del viaje
        - Hour. Hora en la que inicia el viaje en formato `timedelta`
        
    La función devuelve un diccionario con:
        - route. Ruta
        - UpLocationId. Id zona de inicio del viaje
        - DOLocationID. Id zona final del viaje
        - Hour_0. Hora de inicio del viaje
        - Hour_f. Hora en la que finaliza el viaje
        - trip_time. Tiempo del viaje en minutos
        - trip_distance. Distancia recorrida
        - total_amount. Costo del viaje
    '''
    filtro = taxi[taxi['PULocationID'] == UpLocationId]             # Filtramos registros de la zona de inicio del viaje
    destino = np.random.choice(filtro['DOLocationID'])              # Definimos zona de destino del viaje
    filtro = filtro[filtro['DOLocationID'] == destino]              # Actualizamos el filtro
    # Determinar el rango de valores que excluyen outliers
    rango_tiempo = rango(filtro,'time')
    rango_distancia = rango(filtro,'trip_distance')
    # Eliminando registros con datos atipicos
    if len(filtro) != 1:
        filtro = filtro[(filtro['time'] >= rango_tiempo[0]) & (filtro['time'] <= rango_tiempo[1]) & 
                    (filtro['trip_distance'] >= rango_distancia[0]) &
                      (filtro['trip_distance'] <= rango_distancia[1])]
        route = np.random.choice(filtro['route'])
    else:
        filtro = filtro.reset_index()
        route = filtro['route'].iloc[0]
    # Extrayendo datos
    
    distance = np.random.choice(filtro['trip_distance'])
    time = filtro[filtro['trip_distance'] == distance]['time'].mean()
    amount = filtro[filtro['trip_distance'] == distance]['total_amount'].mean()
    try:
        tiempo_espera = timedelta(minutes= 100 * delay(demanda , round(Hour.total_seconds() / 60 , 0), destino))
        # if tiempo_espera >= timedelta(minutes=30):
        #     tiempo_espera = tiempo_espera / 60
    except:
        tiempo_espera = timedelta(minutes= 0)
    Hour_e = Hour + timedelta(hours=0, minutes = time) 
    return {"route":route,"UpLocationId":UpLocationId,"DOLocationID":destino,'wait_time': tiempo_espera, 
            "Hour_0": Hour + tiempo_espera,"Hour_f":Hour_e, "trip_time":time,"trip_distance":distance,
            "total_amount":amount}

In [142]:
def simulacion(n:int,hora_inicio,hora_fin):
    '''
    Función simulacion devuelve un Df con las iteracciones solicitadas por el usuario en el rango de tiempo solicitado.
    - n. Número de simlaciones Montecarlo en valor entero
    - hora_inicio. Hora en la que se inicia la simulación en formato `timedelta`
    - hora_fin. Hora en la que termina la simulación en formato `timedelta`

    Devuelve lista con datos generados por cada iteración y resumen de cada iteración
    
    La lista con datos contiene:
    - route. Ruta
    - UpLocationId. Id zona de inicio del viaje
    - DOLocationID. Id zona final del viaje
    - Hour_0. Hora de inicio del viaje
    - Hour_f. Hora en la que finaliza el viaje
    - trip_time. Tiempo del viaje en minutos
    - trip_distance. Distancia recorrida
    - total_amount. Costo del viaje

    El resumen de las iteraciones contiene la siguiente información:
    - interacion. Numero de iteracion
    - trip_time. El tiempo total de la iteración
    - trip_time_mean. Tiempo promedio por viaje
    - trip_distance. Distancia total recorrida para la iteración
    - trip_mean. Distancia promedio por viaje
    - total_amount. Ingreso total para la iteración
    - amount_mean. Ingreso promedio por viaje
    '''
    lista = []
    resultados = []
    no = 0
    for i in range(0,n):
        idzonaInicio = np.random.choice(taxi['PULocationID'])
        hr_ini = hora_inicio
        data = []
        while hr_ini <= hora_fin:
            a = trip(idzonaInicio,hr_ini)
            data.append(a)
            idzonaInicio = a['DOLocationID']
            hr_ini = a['Hour_f']
        df = pd.DataFrame(data)
        lista.append(df)
    # for j in lista:
    #     res = {"interacion":no,"trip_time":timedelta(minutes=j['trip_time'].sum()),
    #         "trip_time_mean":timedelta(minutes=j['trip_time'].mean()),
    #         "wait_time":j['wait_time'].sum(),
    #         "wait_time_mean":j['wait_time'].mean(),
    #         "trip_distance":j['trip_distance'].sum(),"trip_mean":j['trip_distance'].mean(),
    #         "total_amount":j['total_amount'].sum(),"amount_mean":j['total_amount'].mean()}
    #     resultados.append(res)
    #     no += 1
    # df = pd.DataFrame(resultados)
    return lista #, df

In [121]:
def resumen(dfs):
    resultados = []
    no = 0
    for j in dfs:
        res = {"interacion": no,"trip_time":timedelta(minutes=j['trip_time'].sum()),
            "trip_time_mean":timedelta(minutes=j['trip_time'].mean()),
            "wait_time":j['wait_time'].sum(),
            "wait_time_mean":j['wait_time'].mean(),
            "trip_distance":j['trip_distance'].sum(),"trip_mean":j['trip_distance'].mean(),
            "total_amount":j['total_amount'].sum(),"amount_mean":j['total_amount'].mean(), 
            "CO_emission_total": j['CO2_emission'].sum(), "CO2_emmision_avg":j['CO2_emission'].mean(),
            "Profit": j['Utilidad'].sum()}
        resultados.append(res)
        no += 1
    return pd.DataFrame(resultados).describe()

In [60]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 5
a = simulacion(iteraciones,inicio,fin)

### Simulaciones

Simulación de un solo viaje con zona de inicio aleatorio

In [8]:
inicio = np.random.choice(taxi['PULocationID'])
trip(inicio,timedelta(hours=2,minutes=4))

{'route': 'Midtown South --> Upper East Side North',
 'UpLocationId': 164,
 'DOLocationID': 236,
 'wait_time': datetime.timedelta(seconds=74, microseconds=88842),
 'Hour_0': datetime.timedelta(seconds=7514, microseconds=88842),
 'Hour_f': datetime.timedelta(seconds=8340),
 'trip_time': 15.0,
 'trip_distance': 3.83,
 'total_amount': 23.1}

Simulación de una unica iteración en un horario determinado

In [75]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 1
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteración nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 1 iteración nos brinda:
El recorrido simulado es de 314.71999999999997 millas 
El recorrido promedio por viaje es de 3.085490196078431 millas 
El ingreso simulado es $2678.2344966887813 
El ingreso promedio por viaje es de $26.257200947929228 
Cada viaje duro en promedio 0 days 00:14:28.334100 
Para un total de 1 days 00:36:10.078220 horas 
El total de tiempo ocioso 0 days 00:14:48.086622 
El tiempo promedio de espera entre viajes es de 0 days 00:00:08.706731588


In [85]:
a[0][0][a[0][0]['wait_time'] == timedelta(minutes = 0)]

,route,UpLocationId,DOLocationID,wait_time,Hour_0,Hour_f,trip_time,trip_distance,total_amount
2,Midtown East --> Brooklyn Heights,162,33,0 days,0 days 00:25:13.728070,0 days 00:52:49.728070,27.600000,6.68,45.100000
3,Brooklyn Heights --> Midtown Center,33,161,0 days,0 days 00:52:49.728070,0 days 01:19:09.728070,26.333333,7.10,42.366667
4,Midtown Center --> Lenox Hill West,161,141,0 days,0 days 01:19:09.728070,0 days 01:33:39.728070,14.500000,2.18,21.260833
5,Lenox Hill West --> Upper East Side South,141,237,0 days,0 days 01:33:39.728070,0 days 01:36:34.966165,2.920635,0.31,9.761270
6,Upper East Side South --> Clinton East,237,48,0 days,0 days 01:36:34.966165,0 days 01:50:16.178286,13.686869,1.50,21.063939
...,...,...,...,...,...,...,...,...,...
97,Murray Hill --> Meatpacking/West Village West,170,158,0 days,0 days 22:16:13.559080,0 days 22:31:23.032764,15.157895,2.00,22.797368
98,Meatpacking/West Village West --> Garment Dist...,158,100,0 days,0 days 22:31:23.032764,0 days 22:44:15.532764,12.875000,1.71,20.267500
99,Garment District --> Lenox Hill East,100,140,0 days,0 days 22:44:15.532764,0 days 23:06:10.078219,21.909091,3.30,29.203636
100,Lenox Hill East --> South Ozone Park,140,216,0 days,0 days 23:06:10.078219,0 days 23:47:10.078219,41.000000,14.12,66.800000


Simulación con dos iteraciones

In [90]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 2
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 2 iteraciones nos brinda:
El recorrido simulado es de 261.395 millas 
El recorrido promedio por viaje es de 2.5138368649949108 millas 
El ingreso simulado es $2394.571740938013 
El ingreso promedio por viaje es de $23.011697316595257 
Cada viaje duro en promedio 0 days 00:13:57.896579500 
Para un total de 1 days 00:11:30.032848 horas 
El total de tiempo ocioso 0 days 00:03:42.711234 
El tiempo promedio de espera entre viajes es de 0 days 00:00:02.137378999


Simulación con 10 iteraciones

In [91]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 10
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 10 iteraciones nos brinda:
El recorrido simulado es de 298.30100000000004 millas 
El recorrido promedio por viaje es de 2.9735637005570017 millas 
El ingreso simulado es $3187.661691304046 
El ingreso promedio por viaje es de $31.15018538376001 
Cada viaje duro en promedio 0 days 00:14:24.466142200 
Para un total de 1 days 00:09:31.957588500 horas 
El total de tiempo ocioso 0 days 00:38:49.748924700 
El tiempo promedio de espera entre viajes es de 0 days 00:00:23.999309980


Simulación con 20 iteraciones

In [95]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 20
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 20 iteraciones nos brinda:
El recorrido simulado es de 284.75949999999995 millas 
El recorrido promedio por viaje es de 2.791256488652793 millas 
El ingreso simulado es $3110.293226196339 
El ingreso promedio por viaje es de $29.539771943373275 
Cada viaje duro en promedio 0 days 00:14:09.619837800 
Para un total de 1 days 00:09:38.232835500 horas 
El total de tiempo ocioso 0 days 06:01:38.967401300 
El tiempo promedio de espera entre viajes es de 0 days 00:03:25.422818982


Simulación con 50 iteraciones

In [96]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 50
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 50 iteraciones nos brinda:
El recorrido simulado es de 284.9702 millas 
El recorrido promedio por viaje es de 2.792572669232489 millas 
El ingreso simulado es $2903.4908091389498 
El ingreso promedio por viaje es de $27.926566167010098 
Cada viaje duro en promedio 0 days 00:14:12.833351360 
Para un total de 1 days 00:12:06.561303 horas 
El total de tiempo ocioso 0 days 00:25:12.637160040 
El tiempo promedio de espera entre viajes es de 0 days 00:00:14.698615407


Simulación con 100 iteraciones

In [97]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 100
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 100 iteraciones nos brinda:
El recorrido simulado es de 285.06300000000005 millas 
El recorrido promedio por viaje es de 2.897154589711805 millas 
El ingreso simulado es $2842.151116983946 
El ingreso promedio por viaje es de $27.992385123532006 
Cada viaje duro en promedio 0 days 00:14:58.275712160 
Para un total de 1 days 00:28:10.704787920 horas 
El total de tiempo ocioso 0 days 00:23:16.485828310 
El tiempo promedio de espera entre viajes es de 0 days 00:00:14.185888015


Simulación con 200 iteraciones

In [99]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 200
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 200 iteraciones nos brinda:
El recorrido simulado es de 285.3031 millas 
El recorrido promedio por viaje es de 2.8121597919045715 millas 
El ingreso simulado es $3018.5291169133598 
El ingreso promedio por viaje es de $28.54435887663458 
Cada viaje duro en promedio 0 days 00:14:23.851170715 
Para un total de 1 days 00:14:05.917411270 horas 
El total de tiempo ocioso 0 days 00:51:28.391436775 
El tiempo promedio de espera entre viajes es de 0 days 00:00:31.649970210


Simulación con 500 iteraciones

In [104]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 500
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 500 iteraciones nos brinda:
El recorrido simulado es de 156.84672 millas 
El recorrido promedio por viaje es de 2.8882186165953088 millas 
El ingreso simulado es $1783.3341614144017 
El ingreso promedio por viaje es de $28.828078095428584 
Cada viaje duro en promedio 0 days 00:14:38.254037030 
Para un total de 0 days 13:14:06.401324488 horas


Simulación con 1000 iteraciones

In [105]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 1000
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas \nEl total de tiempo ocioso {a[1]['wait_time'].mean()} \nEl tiempo promedio de espera entre viajes es de {a[1]['wait_time_mean'].mean()}")

La simulacion con 1000 iteraciones nos brinda:
El recorrido simulado es de 206.59527000000003 millas 
El recorrido promedio por viaje es de 3.7342421019383867 millas 
El ingreso simulado es $1688.0670651946282 
El ingreso promedio por viaje es de $28.30094430597562 
Cada viaje duro en promedio 0 days 00:14:24.864502248 
Para un total de 0 days 13:14:49.961863276 horas


Dataframe con el resumen de las 1000 iteraciones

In [121]:
a[1]

,interacion,trip_time,trip_time_mean,trip_distance,trip_mean,total_amount,amount_mean
0,0,0 days 13:08:24.635731,0 days 00:13:49.905890,169.37,2.971404,1998.053453,35.053569
1,1,0 days 13:11:10.137147,0 days 00:11:27.973002,135.53,1.964203,1462.274730,21.192387
2,2,0 days 13:10:22.092379,0 days 00:16:27.960258,158.64,3.305000,1389.489095,28.947689
3,3,0 days 13:03:55.714569,0 days 00:15:04.532972,141.75,2.725962,1350.193030,25.965251
4,4,0 days 13:09:52.646489,0 days 00:16:07.196867,187.42,3.824898,1289.348514,26.313235
...,...,...,...,...,...,...,...
995,995,0 days 13:00:36.882742,0 days 00:12:35.433593,179.06,2.888065,1501.436633,24.216720
996,996,0 days 13:24:07.103452,0 days 00:13:51.846611,145.66,2.511379,1335.804127,23.031106
997,997,0 days 13:02:10.882735,0 days 00:12:13.295043,162.53,2.539531,1455.316371,22.739318
998,998,0 days 13:00:30.537232,0 days 00:16:36.394409,166.55,3.543617,1333.937074,28.381640


Dataframe con el registro de los viajes de una sola iteración

In [120]:
a[0][5]

,route,UpLocationId,DOLocationID,Hour_i,Hour_e,trip_time,trip_distance,total_amount
0,Midtown Center -- Times Sq/Theatre District,161,230,0 days 05:00:00,0 days 05:11:22.758621,11.379310,0.97,17.871897
1,Times Sq/Theatre District -- Penn Station/Madi...,230,186,0 days 05:11:22.758621,0 days 05:21:50.866729,10.468468,1.13,17.152793
2,Penn Station/Madison Sq West -- Union Sq,186,234,0 days 05:21:50.866729,0 days 05:28:31.158700,6.671533,0.92,15.123285
3,Union Sq -- Midtown South,234,164,0 days 05:28:31.158700,0 days 05:35:11.369226,6.670175,0.70,14.663193
4,Midtown South -- Flatiron,164,90,0 days 05:35:11.369226,0 days 05:42:43.133932,7.529412,1.18,16.027647
...,...,...,...,...,...,...,...,...
59,East Harlem North -- Upper East Side South,74,237,0 days 16:55:21.603297,0 days 17:11:21.603297,16.000000,2.70,23.935000
60,Upper East Side South -- Upper East Side South,237,237,0 days 17:11:21.603297,0 days 17:16:58.325230,5.612032,0.70,13.847101
61,Upper East Side South -- Manhattan Valley,237,151,0 days 17:16:58.325230,0 days 17:29:29.325230,12.516667,2.40,22.931167
62,Manhattan Valley -- Midtown Center,151,161,0 days 17:29:29.325230,0 days 17:44:29.325230,15.000000,2.62,20.300000


## Almacenando datos en un archivos parquet

In [148]:
inicio = timedelta(hours=0,minutes=0)
fin = timedelta(hours=23,minutes=59)
iteraciones = 1000
a = simulacion(iteraciones,inicio,fin)
for i in range(0 , 1000):
    no = str(i+301)
    name = '../../Data/Simulation/'+ no +'.parquet'
    a[i].to_parquet(name)

IndexError: list index out of range

In [15]:
dfs = []
for i in range(1,401):
    df = pd.read_parquet(f'../../Data/Simulation/{i}.parquet')
    dfs.append(df)

In [147]:
def interpolacion(Modelo:str, Año:int, electrico:bool,dfs:list,hour_0,hour_f):
    lista = []
    if electrico:
        filtro = electric[(electric['Model'] == Modelo) & (electric['Model year'] == Año)].reset_index()
        co2 = filtro['CO2 emissions (g/km)'].loc[0]
        Mantenimiento = 550
        costo_gas = 0.151 * filtro['City (kWh/100 km)'].loc[0] / 62
    else:
        filtro = fuel[(fuel['Model'] == Modelo) & (fuel['Year'] == Año)].reset_index()
        co2 = filtro['co2'].loc[0]
        Mantenimiento = filtro['Costo_mant_anual'].loc[0]
        costo_gas = filtro['Costo_gasolina_galon'].loc[0]
    for i in dfs:
        i = i[(i['Hour_0'] >= hour_0) & (i['Hour_0'] <= hour_f)]
        i['CO2_emission'] = i['trip_distance'] * co2 * 1.609
        i['Mantenimiento_anual'] = Mantenimiento
        i['Costo_comb'] = i['trip_distance'] * costo_gas
        i['Utilidad'] = i['total_amount'] - i['Costo_comb'] - i['Mantenimiento_anual'] / (365 * i['trip_distance'].sum())
        lista.append(i)
    return resumen(lista)

In [148]:
interpolacion(2019,'Escape FWD',False,dfs,timedelta(hours=5, minutes=0), timedelta(hours=18, minutes=0))

KeyError: 0

In [134]:
fuel.sample()

,Year,Manufacturer,Model,co2,co2TailpipeGpm,fuelType,eng_dscr,VClass,trany,Cap_tanque_comb,Costo_mant_anual,Costo_gasolina_galon,Periodo_mant_preventivo_meses
9988,2019,Ford,Escape FWD,360.0,360.0,Regular,SIDI,Small Sport Utility Vehicle 2WD,Automatic (S6),55.0,750.0,3.62,6
